In [5]:
replicas = 20;

n = 50; % numero de requerimientos de las VM 
dmin = 10; % duracion minima de la VM
dmax = 30; % duracion maxima de la VM
cmin = 25; % requerimiento minimo de capacidad VM
cmax = 50; % requerimiento maximo de capacidad VM
r = n + 1; % para poder ocupar la funcion randi
 
mv = zeros(10 * n, 3); % matriz de 500 filas y 3 columnas, con esta matriz
% generada se trabajara en la heuristica.

for i = 1:replicas
  for j = 1:n
    s = randi([1 r],1,1) - 1;  % start (partidas)
    e = s + randi([dmin dmax],1,1); % end (finalizacion)
    c = randi([cmin cmax],1,1); % c (capacidad demandada)
    posi = (i - 1) * n + j; % asignamos posiciones a la matriz
    % generamos la matriz completa con las 10 replicas:
    mv(posi,1) = s;
    mv(posi,2) = e;
    mv(posi,3) = c;
  end
end

In [6]:
function [prom_serv,prom_arranques] = Heuristica_ACU(n,mv)
  % n es numero de maquinas virtuales y mv es matriz con n filas y 3 columnas
  % que corresponden a s(start), e(end) y c(demand of capacity)

%% 1 Inicializacion

replicas = 20;  

% la idea general consiste en ordenar y priorizar los servidores en funcion
% de su condicion (activo, usado y nuevo)
for rep = 1:replicas
activos = []; % servidores activos
usados = []; % servidores usados
servidores = zeros(n,1);  
nuevos = 100 * ones(n,2); % servidores nuevos
nuevos(:,1) = 1:n;  % existe un total de n servidores disponibles con 
% toda su capacidad

arranques(rep) = 0; % los arranques que queremos minimizar 
num_servidores(rep) = 0; % el numero de servidores para minimizar
disponibilidad = 100 * ones(n,1); % disponibilidad de cada servidor

%% 2 Ordenamos las maquinas virtuales

mv1 = zeros(n,4); % armamos la matriz general y agregamos una columna mas
% que nos entregara el ordenamiento
mv1(:,1) = 1:n;  % marcamos los indices originales de la matriz mv
rango = [(rep - 1) * n + 1:rep * n];
mv1(:,2:4) = mv(rango,:); % matriz con indices, s, e y c.
[~,s] = sort(mv1(:,4),'descend');
mv2 = mv1(s,:); % matriz con capacidad descendiente
[~,s] = sort(mv2(:,2));
mv2 = mv2(s,:); % matriz con start y end ascendente
inicio = zeros(n,3);
inicio(:,1:2) = mv2(:,1:2);
inicio(:,3) = 2;   % asigna baja prioridad a tiempos iniciales

%% 3 Ordenamos los eventos

% El ordenamiento de prioridades de alta y baja est� basado en el supuesto
% de que un evento final de un requerimiento puede dejar a un servidor y 
% dejarlo con toda su disponibilidad justo en el momento que entra otro
% requerimiento. Este suceso no se contabiliza como un arranque, por lo que
% solo en los tiempos finales de los eventos, esto es posible de verificar
% Por lo anterior, consideramos verificar primero los finales con una 
% prioridad alta y los eventos iniciales con prioridad baja. Configuramos 
% los eventos de alta prioridad (finales) en raz�n de la disponibilidad 
% de los servidores y los de baja prioridad (iniciales) en raz�n de la 
% ocupaci�n de los servidores. 

fin = zeros(n,3);
fin(:,1) = mv2(:,1);
fin(:,2) = mv2(:,3);
fin(:,3) = 1; % asigna alta prioridad a tiempos finales
[~,s] = sort(fin(:,2)); % ordena tiempos finales (end) menor a mayor
fin = fin(s,:); % matriz de finales

eventos = zeros(2 * n,3); % crear base matriz final 
eventos(1:n,:) = inicio; % llamar a los eventos de inicio
eventos(n + 1:2 * n,:) = fin; % llamar a los eventos de fin
[~,s] = sort(eventos(:,3)); 
eventos = eventos(s,:);
[~,s] = sort(eventos(:,2));
eventos = eventos(s,:); % declarar matriz con eventos de inicio y fin

%% 4 Tomar primer evento de la lista mientras hayan eventos pendientes

while (size(eventos,1) > 0) % mientras el evento sea distinto de vacio
    
  % identificacion de eventos segun matrices  
  prio = eventos(1,3); % identificar prioridad del evento
  maq = eventos(1,1); % identificar vm
  demanda = mv1(maq,4); % identificar capacidad de demanda de vm
  serv = servidores(maq); % identificar servidor asignado a vm
  
  % identificacion de disponibilidad de cada servidor
  if (serv > 0) 
      disp = disponibilidad(serv);
  endif  
  % activos
  
  % 4.2 Actuar si el evento es fin
  % el concepto de "bandera" se utiliz� solo para identificar cuando
  % deberia existir un arranque, 0: arranque, 1: no hay arranque.
  bandera = 0; % contabilizador de arranques
  if (prio == 1) % si la prioridad es alta del evento
    bandera = 1;
    disponibilidad(serv) = disponibilidad(serv) + demanda;
    % actualizar disponibilidad del servidor activo que corresponde
    % al servidor
    dis2 = disponibilidad(serv);
    % ordenar servidores activos por disponibilidad
    for i = 1:size(activos,1)   
      if (activos(i,1) == serv)
        activos(i,2) = disponibilidad(serv); 
      endif
    endfor
    
    % actuar sobre la lista si la disponibilidad del servidor esta completa
    if disponibilidad(serv) == 100
      [~,s] = sort(activos(:,2),'descend');
      activos = activos(s,:);
    endif
    
    % 4.2.1. Actuar si la disponibilidad esta completa
    if (activos(1,2) == 100) % si quedo 100% disponible
      if size(eventos,1) > 1 % necesito comparar 2 o mas eventos
        % si el proximo evento es de inicio y tiene mismo tiempo actual
        if (eventos(2,2) == eventos(1,2) && eventos(2,3) == 2)
        else
          usados = [usados; activos(1,:)]; % movemos a la lista de usados
          activos(1,:) = []; % actualizo activos
        endif
      endif
      if size(eventos,1) == 1
        usados = [usados; activos(1,:)]; % mover el servidor activo a usado
        activos(1,:) = []; % actualizar activos
      endif               
    endif
  else
    % Si el primero de la lista de los activos tiene capacidad de 
    % absorver la demanda, dejarlo ah�. 
    if size(activos,1) > 0
      if activos(1,2) >= demanda
        % el primer servidor de la lista de activos se asigna a la maquina
        servidores(maq,1) = activos(1,1);
        serv = servidores(maq,1); % actualizamos lista servidores
        % actualizar disponibilidad servidor
        disponibilidad(serv) = disponibilidad(serv) - demanda; 
        activos(1,2) = disponibilidad(serv); % actualizar los activos
        bandera = 1;
      endif 
    endif
    
    % 4.3 Actuar si el evento es inicio, priorizamos servidores usados, si 
    % existen servidores usados con capacidad, entonces toma el usado.
    if bandera == 0
      if size(usados,1) > 0
        servidores(maq,1) = usados(1,1);
        serv = servidores(maq,1);
        disponibilidad(serv) = disponibilidad(serv) - demanda;
        usados(1,2) = disponibilidad(serv); 
        activos = [activos; usados(1,:)]; % movemos a la lista de activos
        usados(1,:) = [];
     
      % si no existen servidores usados, entonces toma los nuevos
      else  
        servidores(maq) = nuevos(1,1);
        serv=servidores(maq);
        % actualizamos disponibilidad
        disponibilidad(serv) = disponibilidad(serv) - demanda;
        nuevos(1,2) = disponibilidad(serv);
        activos = [activos; nuevos(1,:)]; % movemos a la lista de activos
        nuevos(1,:) = [];
      endif
    % contabilizamos los incrementos de arranque por cada replica en 
    % cualquiera de los dos casos
    arranques(rep) = arranques(rep) + 1; 
    endif
  endif
  
  % ordenamos activos de manera descendente con prioridad de mayor
  % disponibilidad
  [~,s] = sort(activos(:,2),'descend');
  activos = activos(s,:); % actualizo servidores activos
  eventos(1,:) = [];
  
endwhile

% 4.4 Recolectar estadisticas
num_servidores(rep) = n - size(nuevos,1);

endfor

%% 5 Presentar las estadisticas y resultados de la heuristica
num_servidores
arranques
prom_serv = mean(num_servidores)
prom_arranques = mean(arranques)

endfunction

Heuristica_ACU(n,mv)

num_servidores =

 Columns 1 through 16:

   11   11   10   11   11   10   12   13   12   13   13   11   12   12   13   11

 Columns 17 through 20:

   11   13   12   12

arranques =

 Columns 1 through 16:

   11   11   10   11   13   11   14   13   13   13   13   12   12   14   13   11

 Columns 17 through 20:

   12   13   12   14

prom_serv = 11.700
prom_arranques = 12.300
ans = 11.700
